In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB
from matplotlib import cm
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline

In [ ]:
# import my functions
import sys

sys.path.append('../../src/')
from ml2gurobi.sklearn import PipelinePredictor

In [ ]:
def peak2d(xx, yy):
    return (3 * (1 - xx) ** 2. * np.exp(- (xx ** 2) - (yy + 1) ** 2)
            - 10 * (xx / 5 - xx ** 4 - yy ** 5) * np.exp(- xx ** 2 - yy ** 2)
            - 1 / 3 * np.exp(-(xx + 1) ** 2 - yy ** 2))

In [ ]:
x = np.arange(-1, 1, 0.01)
y = np.arange(-1, 1, 0.01)
xx, yy = np.meshgrid(x, y)
z = peak2d(xx, yy)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(xx, yy, z, cmap=cm.coolwarm,
                       linewidth=0.01, antialiased=False)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
X = np.concatenate([xx.ravel().reshape(-1, 1),
                    yy.ravel().reshape(-1, 1)
                    ], axis=1)
y = z.ravel()

In [ ]:
# Run our regression
layers = [256, 256]
regression = MLPRegressor(hidden_layer_sizes=layers, activation='relu')
pipe = make_pipeline(regression)
pipe.fit(X=X, y=y)

In [ ]:
X_test = np.random.random((100, 2)) * 2 - 1

In [ ]:
metrics.r2_score(peak2d(X_test[:, 0], X_test[:, 1]), pipe.predict(X_test))

In [ ]:
metrics.max_error(peak2d(X_test[:, 0], X_test[:, 1]), pipe.predict(X_test))

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(xx, yy, pipe.predict(X).reshape(xx.shape),
                       cmap=cm.coolwarm,
                       linewidth=0.01, antialiased=False)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

### Do the optimization model

In [ ]:
optfeat = [0, 1]

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addMVar((len(optfeat)), lb=-1, ub=1, name='x')
y = m.addVar(lb=-GRB.INFINITY, name='y')

m.setObjective(y, gp.GRB.MINIMIZE)

# create transforms to turn scikit-learn pipeline into Gurobi constraints
PipelinePredictor(m, pipe, x, y)

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 100
m.Params.MIPGap = 0.01

In [ ]:
m.optimize()

### Look at the solution

In [ ]:
x.X

Copyright © 2020 Gurobi Optimization, LLC